<a href="https://colab.research.google.com/github/Ekko1zz/project1/blob/main/SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qwen2模型验证

In [1]:
!pip install modelscope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 103.8 MB/s eta 0:00:00


In [2]:
from modelscope import AutoModelForCausalLM,AutoTokenizer
import torch

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-0.5B-Instruct")

2025-04-04 16:52:28,712 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-04-04 16:52:29,121 - modelscope - INFO - Got 10 files, start to download ...


Processing 10 items:   0%|          | 0.00/10.0 [00:00<?, ?it/s]

2025-04-04 16:55:43,739 - modelscope - INFO - Download model 'qwen/Qwen2-0.5B-Instruct' successfully.
2025-04-04 16:55:43,740 - modelscope - INFO - Creating symbolic link [/root/.cache/modelscope/hub/models/qwen/Qwen2-0.5B-Instruct].
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


2025-04-04 16:55:49,481 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-04-04 16:55:49,907 - modelscope - INFO - Target directory already exists, skipping creation.


In [3]:
print(tokenizer)

Qwen2TokenizerFast(name_or_path='/root/.cache/modelscope/hub/models/qwen/Qwen2-0___5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [18]:
def chat(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print(text)
    print()
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [19]:
prompt = "你是谁？"
chat(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你是谁？<|im_end|>
<|im_start|>assistant




'我是来自阿里云的大规模语言模型，我叫通义千问。'

In [20]:
# 训练数据预处理方法
def preprocess(tokenizer,batch_messages):
    input_list=[]
    target_list=[]

    im_start=tokenizer('<|im_start|>').input_ids
    im_end=tokenizer('<|im_end|>').input_ids
    newline=tokenizer('\n').input_ids
    pad=tokenizer('<|endoftext|>').input_ids
    ignore=[-100]

    for group in batch_messages:
        input_ids=[]
        target_ids=[]
        for msg in group:
            role=tokenizer(msg['role']).input_ids
            content=tokenizer(msg['content']).input_ids
            if msg['role'] in ['system','user']:
                ignore_parts=role+newline+content
                input_ids+=im_start+ignore_parts+im_end+newline
                target_ids+=im_start+ignore*len(ignore_parts)+im_end+newline
            else:
                ignore_parts=role+newline
                input_ids+=im_start+ignore_parts+content+im_end+newline
                target_ids+=im_start+ignore*len(ignore_parts)+content+im_end+newline
        input_list.append(input_ids)
        target_list.append(target_ids)

    # padding
    max_len=max([len(ids) for ids in input_list])
    for input_ids,target_ids in zip(input_list,target_list):
        input_ids+=pad*(max_len-len(input_ids))
        target_ids+=ignore*(max_len-len(target_ids))
    batch_input_ids=torch.tensor(input_list,dtype=torch.long)
    batch_target_ids=torch.tensor(target_list,dtype=torch.long)
    batch_mask=batch_input_ids.ne(pad[0]).type(torch.long)
    return batch_input_ids,batch_target_ids,batch_mask

# 训练数据测试

In [34]:
prompt = "2+2等于几"
messages = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '2+2等于5。'},
    ],
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '2+2等于5。'},
    ]
]

# model.eval()
optimizer=torch.optim.SGD(model.parameters())
optimizer.zero_grad()

batch_input_ids,batch_target_ids,batch_mask=preprocess(tokenizer,messages)
model_outputs=model(batch_input_ids.to(device))

output_tokens=model_outputs.logits.argmax(dim=-1)

logits=model_outputs.logits[:,:-1,:]
targets=batch_target_ids[:,1:].to(device)
print('logits:',logits.shape) # 模型输出
print('targets:',targets.shape) # 拟合目标

from torch.nn import CrossEntropyLoss

# 损失
loss_fn=CrossEntropyLoss()
loss=loss_fn(logits.reshape(-1,logits.size(2)),targets.reshape(-1))
print('loss:',loss)

# 优化器
# optimizer=torch.optim.SGD(model.parameters())
# optimizer.zero_grad()

# 求梯度
loss.backward()

# 梯度下降
optimizer.step()

logits: torch.Size([2, 31, 151936])
targets: torch.Size([2, 31])
loss: tensor(1.0859, device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<NllLossBackward0>)


In [35]:
model.eval()


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [36]:
model.eval()

chat('2+2等于4么')

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
2+2等于4么<|im_end|>
<|im_start|>assistant




'2+2等于5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+2=5。2+'

In [30]:
prompt = "2+2等于几"
messages = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '2+2等于5。'},
    ],
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '2+2等于5。'},
    ]
]

model.eval()

batch_input_ids,batch_target_ids,batch_mask=preprocess(tokenizer,messages)
print(batch_input_ids)
print(batch_input_ids.shape)
print(batch_target_ids)
print(batch_target_ids.shape)

print(batch_mask)
print(batch_mask.shape)
print()
model_outputs=model(batch_input_ids.to(device))
print(model_outputs)


tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,     17,     10,     17, 107106,
          99195, 151645,    198, 151644,  77091,    198,     17,     10,     17,
         107106,     20,   1773, 151645,    198],
        [151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,     17,     10,     17, 107106,
          99195, 151645,    198, 151644,  77091,    198,     17,     10,     17,
         107106,     20,   1773, 151645,    198]])
torch.Size([2, 32])
tensor([[151644,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
         151645,    198, 151644,   -100,   -100,   -100,   -100,   -100,   -100,
           -100, 151645,    198, 151644,   -100,   -100,     17,     10,     17,
         107106,     20,   1773, 151645,    198],
        [151644,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
         151645,   

In [31]:
print(model_outputs.logits.shape)
print(model_outputs.logits)

torch.Size([2, 32, 151936])
tensor([[[ 2.3594,  2.6250,  2.9688,  ..., -3.3125, -3.3125, -3.3125],
         [ 0.6094,  1.8828,  3.7031,  ..., -4.5000, -4.5000, -4.5000],
         [ 8.2500,  4.7188, 10.9375,  ..., -3.4688, -3.4688, -3.4688],
         ...,
         [ 6.8750, 10.2500,  7.7500,  ..., -3.3906, -3.3906, -3.3906],
         [ 4.2812,  6.0312,  2.0156,  ..., -3.7344, -3.7188, -3.7188],
         [ 9.1250,  7.7500,  4.5312,  ..., -3.5625, -3.5625, -3.5625]],

        [[ 2.3594,  2.6250,  2.9688,  ..., -3.3125, -3.3125, -3.3125],
         [ 0.6094,  1.8828,  3.7031,  ..., -4.5000, -4.5000, -4.5000],
         [ 8.2500,  4.7188, 10.9375,  ..., -3.4688, -3.4688, -3.4688],
         ...,
         [ 6.8750, 10.2500,  7.7500,  ..., -3.3906, -3.3906, -3.3906],
         [ 4.2812,  6.0312,  2.0156,  ..., -3.7344, -3.7188, -3.7188],
         [ 9.1250,  7.7500,  4.5312,  ..., -3.5625, -3.5625, -3.5625]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)


In [26]:
output_tokens=model_outputs.logits.argmax(dim=-1)

logits=model_outputs.logits[:,:-1,:]
targets=batch_target_ids[:,1:].to(device)
print('logits:',logits.shape) # 模型输出
print('targets:',targets.shape) # 拟合目标

logits: torch.Size([2, 31, 151936])
targets: torch.Size([2, 31])
